In [6]:
import geopandas as gpd
import re
from pyproj import Proj
import pandas as pd

In [10]:
base = gpd.read_file('../dados/arborizacao/SIRGAS_SHP_arvore_.shp')

In [11]:
base.head()

,av_subpref,av_distr,av_sisgau,av_codlog,av_codlogd,av_base,geometry
0,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329187.277 7387899.634)
1,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329181.76 7387904.182)
2,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329172.255 7387912.306)
3,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329159.687 7387922.7)
4,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329164.589 7387918.805)


In [12]:
df_geral = gpd.GeoDataFrame(base)

In [14]:
df_geral['geometry'] = df_geral['geometry'].astype(str)

C:\Users\prado\AppData\Local\Temp\ipykernel_27544\4291180805.py:1: UserWarning: Geometry column does not contain geometry.
  df_geral['geometry'] = df_geral['geometry'].astype(str)


In [15]:
# Definindo o sistema de coordenadas UTM, zona 23S
utm_proj = Proj(proj='utm', zone=23, south=True, ellps='WGS84')

# Função para extrair coordenadas e converter para latitude/longitude
def utm_para_latlon(geometry):
    # Usa regex para capturar as coordenadas se estiverem no formato de texto 'POINT'
    match = re.match(r'POINT \((\d+\.\d+) (\d+\.\d+)\)', geometry)
    if match:
        easting = float(match.group(1))
        northing = float(match.group(2))
        # Conversão de UTM para Lat/Lon
        lon, lat = utm_proj(easting, northing, inverse=True)
        return lat, lon
    return None, None  # Retorna valores nulos caso a expressão não corresponda

# Supondo que 'geometry' seja uma coluna de strings com a geometria no formato 'POINT (E N)'
df_geral['Latitude'], df_geral['Longitude'] = zip(*df_geral['geometry'].apply(utm_para_latlon))

In [16]:
df_geral

,av_subpref,av_distr,av_sisgau,av_codlog,av_codlogd,av_base,geometry,Latitude,Longitude
0,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329187.277133 7387899.634205),-23.610464,-46.674227
1,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329181.759825 7387904.18161),-23.610423,-46.674281
2,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329172.254511 7387912.305917),-23.610348,-46.674373
3,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329159.686893 7387922.700328),-23.610253,-46.674495
4,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329164.5895 7387918.804724),-23.610289,-46.674447
...,...,...,...,...,...,...,...,...,...
652380,VILA MARIANA,MOEMA,None,07708,9,ORTOFOTOS EMPLASA,POINT (329191.427242 7387882.223284),-23.610622,-46.674189
652381,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329188.237835 7387895.3629),-23.610503,-46.674218
652382,VILA MARIANA,MOEMA,None,07708,9,ORTOFOTOS EMPLASA,POINT (329184.956932 7387890.383093),-23.610548,-46.674251
652383,VILA MARIANA,MOEMA,None,38836,0,ORTOFOTOS EMPLASA,POINT (329158.291306 7387860.422749),-23.610815,-46.674516


In [17]:
df_geral.to_csv('../dados/localizacao_arborizacao_viaria.csv', index=False)